In [2]:
# import packages

import numpy as np
import pandas as pd

import glob
import os
import warnings
import snowflake.connector

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
# load pre-calculated clusters (to-do: recalculate)

In [2]:
df_cluster = pd.read_pickle('df_cluster_master.pkl')
print(df_cluster.shape)
print(df_cluster.columns.values)
df_cluster.head()

(2115, 5)
['Zip' 'Avg_3_Clusters' 'Avg_4_Clusters' 'Avg_5_Clusters' 'SEL_Cluster']


,Zip,Avg_3_Clusters,Avg_4_Clusters,Avg_5_Clusters,SEL_Cluster
0,57551,AVG-3.1,AVG-4.1,AVG-5.4,SEL-4
1,57702,AVG-3.1,AVG-4.4,AVG-5.3,SEL-2
2,57714,AVG-3.1,AVG-4.1,AVG-5.4,SEL-4
3,57716,AVG-3.1,AVG-4.1,AVG-5.4,SEL-4
4,57752,AVG-3.1,AVG-4.1,AVG-5.4,SEL-4


In [4]:
df_cluster.to_csv('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\zip_to_cluster.csv', index=False)
df_cluster.to_pickle('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\zip_to_cluster.pkl')

In [5]:
# Load sales data - can skip if pickle already saved

In [ ]:
con = snowflake.connector.connect(
    user="ST000421", #You can get it by executing in UI: desc user <username>;
    account="swire.east-us-2.azure", #Add all of the account-name between https:// and snowflakecomputing.com
    authenticator="externalbrowser",
    warehouse='SWIRE_WH_ELDAMO_M'
)
cur = con.cursor()

In [ ]:
from datetime import date
cur_first_date = date.today().replace(day=1)
print(cur_first_date)
ly_first_date = cur_first_date.replace(year = cur_first_date.year-1) 
print(ly_first_date)

In [ ]:
query_swire_sales='select  \
                        Customer.CUSTOMER_NUMBER \
                        , Customer.CUSTOMER_NAME    \
                        , Customer.KEY_ACCOUNT_PLAN_DESCRIPTION_CP3 \
                        , Customer.NAME_2   \
                        , Customer.ADDRESS_CITY \
                        , Customer.ADDRESS_ZIP_CODE \
                        , Customer.ADDRESS_STATE    \
                        , Customer.COUNTY   \
                        , Customer.DIVISION \
                        , Customer.SEARCH_TERM  \
                        , Customer.CUSTOMER_TRADE_CHANNEL_DESCRIPTION   \
                        , Customer.SUPER_CHANNEL_DEFINITION     \
                        , Sales."Material Number"   \
                        , Material."MATERIAL_DESCRIPTION"   \
                        , Material."SWIRE_PACKAGE_CATEGORY_DESCRIPTION" \
                        , Material."PACK_SIZE_SALES_UNIT_DESCRIPTION"   \
                        , Material."PACK_TYPE_DESC" \
                        , Material.CS_UPC   \
                        , Material.EA_UPC   \
                        , Material.EAN_UPC  \
                        , Material.CALORIE_CAT_DESC  \
                        , Material.IC_FC_ATTRIBUTE  \
                        , Material.FLAVOUR_DESC  \
                        , Material.BEV_PRODUCT_DESC  \
                        , Material.BRAND_DESC  \
                        , PROC_TYP.procurement_type \
                        , sum(Sales."GrossProfitDeadNet") as Tot_GrossProfitDeadNet \
                        , sum(Sales."GrossProfitDeadNet")/sum(Sales."PhysicalVolume") as PerCase_GrossProfitDeadNet \
                        , sum(Sales."PhysicalVolume") as Tot_Volume \
                        , sum(Sales."Wholesale") as Tot_Wholesale   \
                        , sum(Sales."Discounts") as Tot_Discounts   \
                        , sum(Sales."NSI") as Tot_NSI   \
                        , sum(Sales."DeadNet") as Tot_DeadNet   \
                        , sum(Sales."TotalCOGS") as Tot_TotalCOGS   \
                        , sum(Sales."GrossProfit") as Tot_GrossProfit   \
                    from DB_SWIRE_BI_P_EDW.TRANSFORMED.FACT_COPA_MARS_SALES as Sales \
                    inner join DB_SWIRE_BI_P_EDW.TRANSFORMED.DIM_MDM_CUSTOMER_MASTER as Customer on (Sales."Ship To" = Customer."CUSTOMER_NUMBER")  \
                    inner join "DB_SWIRE_BI_P_EDW"."TRANSFORMED"."DIM_MDM_MATERIAL_MASTER" as Material on (Sales."Material Number" = Material."MATERIAL_ID")    \
                    left join (select distinct T01.material_id,procurement_type \
                                from DB_BI_P_EDW.GENERAL_USE_BAS.BAS_MDM_MATERIAL_PLANT as T01  \
                                inner join DB_BI_P_EDW.GENERAL_USE_BAS.BAS_MDM_MATERIAL_SALES T02 on T01.material_sk = T02.MATERIAL_SK  \
                                where 1 = 1 \
                                and procurement_type = \'E\'  \
                                ) \
                    as PROC_TYP on (Material."MATERIAL_ID"=PROC_TYP.material_id)    \
                    where \
                    Sales."Original Date" >= date(\'{0}\')  \
                    and Sales."Original Date" < date(\'{1}\')  \
                    and Material."MATERIAL_TYPE" in (\'ZFER\')    \
                    and Material."PHYSICAL_STATE_DESC" = \'PACKAGED FINISHED DRINK (RTD)\'    \
                    and Sales."PhysicalVolume" >0   \
                    and Sales."Value Type"=\'10\' \
                    group by    \
                        Customer.CUSTOMER_NUMBER \
                        , Customer.CUSTOMER_NAME    \
                        , Customer.KEY_ACCOUNT_PLAN_DESCRIPTION_CP3 \
                        , Customer.NAME_2   \
                        , Customer.ADDRESS_CITY \
                        , Customer.ADDRESS_ZIP_CODE \
                        , Customer.ADDRESS_STATE    \
                        , Customer.COUNTY   \
                        , Customer.DIVISION \
                        , Customer.SEARCH_TERM  \
                        , Customer.CUSTOMER_TRADE_CHANNEL_DESCRIPTION   \
                        , Customer.SUPER_CHANNEL_DEFINITION     \
                        , Sales."Material Number"   \
                        , Material."MATERIAL_DESCRIPTION"   \
                        , Material."SWIRE_PACKAGE_CATEGORY_DESCRIPTION" \
                        , Material."PACK_SIZE_SALES_UNIT_DESCRIPTION"   \
                        , Material."PACK_TYPE_DESC" \
                        , Material.CS_UPC   \
                        , Material.EA_UPC   \
                        , Material.EAN_UPC  \
                        , Material.CALORIE_CAT_DESC  \
                        , Material.IC_FC_ATTRIBUTE  \
                        , Material.FLAVOUR_DESC  \
                        , Material.BEV_PRODUCT_DESC  \
                        , Material.BRAND_DESC  \
                        , PROC_TYP.procurement_type \
                '.format(ly_first_date,cur_first_date)
print(query_swire_sales)

In [ ]:
cur.execute(query_swire_sales)
df_swire_sales=cur.fetch_pandas_all()
df_swire_sales.head()

In [ ]:
con.close()
cur.close()

In [ ]:
df_swire_sales['Zip']=df_swire_sales['ADDRESS_ZIP_CODE'].str.slice(0,5)
df_swire_sales.to_pickle('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\df_swire_sales_2022.pkl')

In [ ]:
# Calculate product attributes

In [3]:
df_swire_sales = pd.read_pickle('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\df_swire_sales_2022.pkl')
print(df_swire_sales.shape)
print(df_swire_sales.columns.values)
df_swire_sales.head()

(3465616, 36)
['CUSTOMER_NUMBER' 'CUSTOMER_NAME' 'KEY_ACCOUNT_PLAN_DESCRIPTION_CP3'
 'NAME_2' 'ADDRESS_CITY' 'ADDRESS_ZIP_CODE' 'ADDRESS_STATE' 'COUNTY'
 'DIVISION' 'SEARCH_TERM' 'CUSTOMER_TRADE_CHANNEL_DESCRIPTION'
 'SUPER_CHANNEL_DEFINITION' 'Material Number' 'MATERIAL_DESCRIPTION'
 'SWIRE_PACKAGE_CATEGORY_DESCRIPTION' 'PACK_SIZE_SALES_UNIT_DESCRIPTION'
 'PACK_TYPE_DESC' 'CS_UPC' 'EA_UPC' 'EAN_UPC' 'CALORIE_CAT_DESC'
 'IC_FC_ATTRIBUTE' 'FLAVOUR_DESC' 'BEV_PRODUCT_DESC' 'BRAND_DESC'
 'PROCUREMENT_TYPE' 'TOT_GROSSPROFITDEADNET' 'PERCASE_GROSSPROFITDEADNET'
 'TOT_VOLUME' 'TOT_WHOLESALE' 'TOT_DISCOUNTS' 'TOT_NSI' 'TOT_DEADNET'
 'TOT_TOTALCOGS' 'TOT_GROSSPROFIT' 'Zip']


,CUSTOMER_NUMBER,CUSTOMER_NAME,KEY_ACCOUNT_PLAN_DESCRIPTION_CP3,NAME_2,ADDRESS_CITY,ADDRESS_ZIP_CODE,ADDRESS_STATE,COUNTY,DIVISION,SEARCH_TERM,...,TOT_GROSSPROFITDEADNET,PERCASE_GROSSPROFITDEADNET,TOT_VOLUME,TOT_WHOLESALE,TOT_DISCOUNTS,TOT_NSI,TOT_DEADNET,TOT_TOTALCOGS,TOT_GROSSPROFIT,Zip
0,0600078143,WALMART SC #4252,WALMART,,FLAGSTAFF,86004,AZ,COCONINO,AZ,WALMART SC,...,617.5400000000000000,3.4889265536723164,177.0000000000000000,4248.0000000000000000,1527.8300000000000000,2720.1700000000000000,2720.1700000000000000,2295.3000000000000000,424.8700000000000000,86004
1,0600078143,WALMART SC #4252,WALMART,,FLAGSTAFF,86004,AZ,COCONINO,AZ,WALMART SC,...,1357.6800000000000000,6.8225125628140704,199.0000000000000000,7164.0000000000000000,3534.2400000000000000,3629.7600000000000000,2824.7500000000000000,1542.6900000000000000,2087.0700000000000000,86004
2,0600078143,WALMART SC #4252,WALMART,,FLAGSTAFF,86004,AZ,COCONINO,AZ,WALMART SC,...,4458.8600000000000000,4.8624427480916031,917.0000000000000000,44016.0000000000000000,21714.5600000000000000,22301.4400000000000000,12797.6800000000000000,8829.6400000000000000,13471.8000000000000000,86004
3,0600078143,WALMART SC #4252,WALMART,,FLAGSTAFF,86004,AZ,COCONINO,AZ,WALMART SC,...,118.0600000000000000,2.2703846153846154,52.0000000000000000,1872.0000000000000000,443.0400000000000000,1428.9600000000000000,1062.4900000000000000,944.4300000000000000,484.5300000000000000,86004
4,0600078143,WALMART SC #4252,WALMART,,FLAGSTAFF,86004,AZ,COCONINO,AZ,WALMART SC,...,150.6800000000000000,3.0751020408163265,49.0000000000000000,1372.0000000000000000,785.4800000000000000,586.5200000000000000,586.5200000000000000,507.4500000000000000,79.0700000000000000,86004


In [5]:
cols = ['CUSTOMER_NUMBER', 'KEY_ACCOUNT_PLAN_DESCRIPTION_CP3',
 'SUPER_CHANNEL_DEFINITION', 'MATERIAL_DESCRIPTION', 'PACK_SIZE_SALES_UNIT_DESCRIPTION',
 'PACK_TYPE_DESC', 'CS_UPC', 'EA_UPC', 'EAN_UPC', 'CALORIE_CAT_DESC',
 'IC_FC_ATTRIBUTE', 'FLAVOUR_DESC', 'BEV_PRODUCT_DESC', 'BRAND_DESC',
 'PROCUREMENT_TYPE', 'TOT_GROSSPROFITDEADNET', 
 'TOT_VOLUME', 'Zip']
df_sales_slim = df_swire_sales[cols]
gcols = ['CUSTOMER_NUMBER', 'KEY_ACCOUNT_PLAN_DESCRIPTION_CP3',
 'SUPER_CHANNEL_DEFINITION', 'MATERIAL_DESCRIPTION', 'PACK_SIZE_SALES_UNIT_DESCRIPTION',
 'PACK_TYPE_DESC', 'CS_UPC', 'EA_UPC', 'EAN_UPC', 'CALORIE_CAT_DESC',
 'IC_FC_ATTRIBUTE', 'FLAVOUR_DESC', 'BEV_PRODUCT_DESC', 'BRAND_DESC',
 'PROCUREMENT_TYPE', 'Zip']
print(df_sales_slim.shape)
df_sales_slim = df_sales_slim.groupby(gcols)[['TOT_GROSSPROFITDEADNET', 'TOT_VOLUME']].sum().reset_index()
df_sales_slim.to_csv('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\slim_sales.csv', index=False)
df_sales_slim.to_pickle('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\slim_sales.pkl')
print(df_sales_slim.shape)

(3465616, 18)
(1434155, 18)


In [7]:
# Create UPC and Zip Code level summary
grouping = ['CS_UPC','EA_UPC', 'EAN_UPC','SUPER_CHANNEL_DEFINITION','Zip']
adding = ['TOT_VOLUME','TOT_DEADNET','TOT_GROSSPROFITDEADNET','TOT_NSI']
df_swire_upc_summary = df_swire_sales.groupby(grouping)[adding].sum().reset_index()
renaming = {'TOT_VOLUME':'SUM_VOLUME','TOT_DEADNET':'SUM_DEADNET',
            'TOT_GROSSPROFITDEADNET':'SUM_GROSSPROFITDEADNET','TOT_NSI':'SUM_NSI'}
df_swire_upc_summary = df_swire_upc_summary.rename(columns=renaming)
df_swire_upc_summary.head()

,CS_UPC,EA_UPC,EAN_UPC,SUPER_CHANNEL_DEFINITION,Zip,SUM_VOLUME,SUM_DEADNET,SUM_GROSSPROFITDEADNET,SUM_NSI
0,21136010381,21136110449,21136010381,ALL OTHER,83301,2.0000000000000000,5.0000000000000000,-12.4300000000000000,5.0000000000000000
1,21136010381,21136110449,21136010381,ALL OTHER,84020,17.0000000000000000,612.0000000000000000,486.6900000000000000,612.0000000000000000
2,21136010381,21136110449,21136010381,ALL OTHER,84120,120.0000000000000000,2073.6000000000000000,1180.3900000000000000,2073.6000000000000000
3,21136010381,21136110449,21136010381,ALL OTHER,97230,120.0000000000000000,360.0000000000000000,-513.8400000000000000,360.0000000000000000
4,21136010381,21136110449,21136010381,ALL OTHER,97301,120.0000000000000000,360.0000000000000000,-513.8400000000000000,360.0000000000000000


In [8]:
# Create UPC to Mat desc mapping
df_material= df_swire_sales[['MATERIAL_DESCRIPTION','BEV_PRODUCT_DESC','BRAND_DESC','SWIRE_PACKAGE_CATEGORY_DESCRIPTION','PACK_SIZE_SALES_UNIT_DESCRIPTION', 'PACK_TYPE_DESC', 'CS_UPC','EA_UPC', 'EAN_UPC','CALORIE_CAT_DESC', 'IC_FC_ATTRIBUTE','FLAVOUR_DESC']].drop_duplicates()
print(df_material.shape)
df_upc_mat_mapping=df_material.groupby(['CS_UPC','EA_UPC', 'EAN_UPC']).first().reset_index()
print(df_upc_mat_mapping.shape)

(994, 12)
(779, 12)


In [9]:
# Join Mat description to UPC level summary data
print(df_swire_upc_summary.shape)
df_swire_upc_summary2=pd.merge(left=df_swire_upc_summary,right=df_upc_mat_mapping,on=['CS_UPC','EA_UPC', 'EAN_UPC'],how="inner")
print(df_swire_upc_summary2.shape)

(1425637, 9)
(1425637, 18)


In [10]:
# Calculate VOL for entire AU- This is the benchmark to compare against
df_upc_au=df_swire_upc_summary2[['CS_UPC', 'EA_UPC', 'EAN_UPC','SUPER_CHANNEL_DEFINITION','SUM_VOLUME']]
df_upc_au_vol_aggregated=df_upc_au.groupby(['CS_UPC', 'EA_UPC', 'EAN_UPC','SUPER_CHANNEL_DEFINITION'],as_index=False).agg(AU_SUM_VOLUME=pd.NamedAgg(column='SUM_VOLUME',aggfunc=lambda x: x.sum()),
                                                                                               AU_ROW_COUNTS=pd.NamedAgg(column='EA_UPC',aggfunc=lambda x: x.count()))

In [11]:
# Calculate VOL for Zips- This is the data that will be compared against the benchmark
df_upc_strata_zip=df_swire_upc_summary2[['SUPER_CHANNEL_DEFINITION','Zip','CS_UPC', 'EA_UPC', 'EAN_UPC','SUM_VOLUME','SUM_GROSSPROFITDEADNET']]
df_upc_strata_zip_vol_aggregated=df_upc_strata_zip.groupby(['SUPER_CHANNEL_DEFINITION','Zip','CS_UPC', 'EA_UPC', 'EAN_UPC'],as_index=False).agg(STRATA_SUM_VOLUME=pd.NamedAgg(column='SUM_VOLUME',aggfunc=lambda x: x.sum()),
                                                                                                                                                    STRATA_SUM_GROSSPROFITDEADNET=pd.NamedAgg(column='SUM_GROSSPROFITDEADNET',aggfunc=lambda x: x.sum()),
                                                                                                                                                    STRATA_ROW_COUNTS=pd.NamedAgg(column='EA_UPC',aggfunc=lambda x: x.count()))

In [12]:
df_swire_grouped = df_swire_upc_summary2.merge(df_upc_au_vol_aggregated, 
                                on = ['CS_UPC', 'EA_UPC', 'EAN_UPC','SUPER_CHANNEL_DEFINITION'], how='inner')
df_swire_grouped = df_swire_grouped.merge(df_upc_strata_zip_vol_aggregated,
                                on = ['SUPER_CHANNEL_DEFINITION','Zip','CS_UPC', 'EA_UPC', 'EAN_UPC'], how='inner')
print(df_swire_grouped.shape)
print(df_swire_grouped.columns.values)
df_swire_grouped.head()

(1425637, 23)
['CS_UPC' 'EA_UPC' 'EAN_UPC' 'SUPER_CHANNEL_DEFINITION' 'Zip' 'SUM_VOLUME'
 'SUM_DEADNET' 'SUM_GROSSPROFITDEADNET' 'SUM_NSI' 'MATERIAL_DESCRIPTION'
 'BEV_PRODUCT_DESC' 'BRAND_DESC' 'SWIRE_PACKAGE_CATEGORY_DESCRIPTION'
 'PACK_SIZE_SALES_UNIT_DESCRIPTION' 'PACK_TYPE_DESC' 'CALORIE_CAT_DESC'
 'IC_FC_ATTRIBUTE' 'FLAVOUR_DESC' 'AU_SUM_VOLUME' 'AU_ROW_COUNTS'
 'STRATA_SUM_VOLUME' 'STRATA_SUM_GROSSPROFITDEADNET' 'STRATA_ROW_COUNTS']


,CS_UPC,EA_UPC,EAN_UPC,SUPER_CHANNEL_DEFINITION,Zip,SUM_VOLUME,SUM_DEADNET,SUM_GROSSPROFITDEADNET,SUM_NSI,MATERIAL_DESCRIPTION,...,PACK_SIZE_SALES_UNIT_DESCRIPTION,PACK_TYPE_DESC,CALORIE_CAT_DESC,IC_FC_ATTRIBUTE,FLAVOUR_DESC,AU_SUM_VOLUME,AU_ROW_COUNTS,STRATA_SUM_VOLUME,STRATA_SUM_GROSSPROFITDEADNET,STRATA_ROW_COUNTS
0,21136010381,21136110449,21136010381,ALL OTHER,83301,2.0000000000000000,5.0000000000000000,-12.4300000000000000,5.0000000000000000,12Z GL 12PX1 CD_TOP CHC MIN WTRTWSTOFLIM,...,12 OZ 12-Pk 12,Glass Bottle - Other,LOW CALORIE,FC,LEMON LIME WATER,466.0000000000000000,7,2.0000000000000000,-12.4300000000000000,1
1,21136010381,21136110449,21136010381,ALL OTHER,84020,17.0000000000000000,612.0000000000000000,486.6900000000000000,612.0000000000000000,12Z GL 12PX1 CD_TOP CHC MIN WTRTWSTOFLIM,...,12 OZ 12-Pk 12,Glass Bottle - Other,LOW CALORIE,FC,LEMON LIME WATER,466.0000000000000000,7,17.0000000000000000,486.6900000000000000,1
2,21136010381,21136110449,21136010381,ALL OTHER,84120,120.0000000000000000,2073.6000000000000000,1180.3900000000000000,2073.6000000000000000,12Z GL 12PX1 CD_TOP CHC MIN WTRTWSTOFLIM,...,12 OZ 12-Pk 12,Glass Bottle - Other,LOW CALORIE,FC,LEMON LIME WATER,466.0000000000000000,7,120.0000000000000000,1180.3900000000000000,1
3,21136010381,21136110449,21136010381,ALL OTHER,97230,120.0000000000000000,360.0000000000000000,-513.8400000000000000,360.0000000000000000,12Z GL 12PX1 CD_TOP CHC MIN WTRTWSTOFLIM,...,12 OZ 12-Pk 12,Glass Bottle - Other,LOW CALORIE,FC,LEMON LIME WATER,466.0000000000000000,7,120.0000000000000000,-513.8400000000000000,1
4,21136010381,21136110449,21136010381,ALL OTHER,97301,120.0000000000000000,360.0000000000000000,-513.8400000000000000,360.0000000000000000,12Z GL 12PX1 CD_TOP CHC MIN WTRTWSTOFLIM,...,12 OZ 12-Pk 12,Glass Bottle - Other,LOW CALORIE,FC,LEMON LIME WATER,466.0000000000000000,7,120.0000000000000000,-513.8400000000000000,1


In [13]:
# Calculate product indices

In [14]:
# Calculate percentages within each zip and superchannel
df = df_swire_grouped.copy()
dfg = df.groupby(['Zip','SUPER_CHANNEL_DEFINITION'])['STRATA_SUM_VOLUME'].sum().reset_index(name='ZIP_SC_GROUP')
df = df.merge(dfg, on=['Zip','SUPER_CHANNEL_DEFINITION'], how='inner')
df['ZIP_SC_PERCENTAGE'] = df['STRATA_SUM_VOLUME']/df['ZIP_SC_GROUP']

In [15]:
# Calculate percentages within each superchannel, across Swire footprint zips -- is this already AU_SUM_VOLUME btw? 
dfga1 = df.groupby(['SUPER_CHANNEL_DEFINITION'])['STRATA_SUM_VOLUME'].sum().reset_index(name='FOOTPRINT_SC_GROUP')
dfga2 = df.groupby(['CS_UPC', 'EA_UPC', 'EAN_UPC', 'SUPER_CHANNEL_DEFINITION'])['STRATA_SUM_VOLUME'].sum().reset_index(name='FOOTPRINT_SC_PROD')
dfga3 = dfga2.merge(dfga1, on=['SUPER_CHANNEL_DEFINITION'], how='inner')
dfga3['FOOTPRINT_SC_PERCENTAGE'] = dfga3['FOOTPRINT_SC_PROD']/dfga3['FOOTPRINT_SC_GROUP']

In [16]:
# Calculate index as percentage within this zip/superchannel minus percentage within superchannel across Swire footprint + 100
df = df.merge(dfga3, on=['CS_UPC', 'EA_UPC', 'EAN_UPC', 'SUPER_CHANNEL_DEFINITION'], how='inner').reset_index(drop=True)
df = df.sort_values(by=['Zip','SUPER_CHANNEL_DEFINITION']).reset_index(drop=True)
df['PRODUCT_INDEX'] =100*(df['ZIP_SC_PERCENTAGE']-df['FOOTPRINT_SC_PERCENTAGE'])+100
df['INDEX_SUMPROD'] = df['PRODUCT_INDEX']*df['STRATA_SUM_VOLUME']

In [17]:
df['PRODUCT_INDEX'] = df['PRODUCT_INDEX'].astype(float)
df['STRATA_SUM_VOLUME'] = df['STRATA_SUM_VOLUME'].astype(float)
df['STRATA_SUM_GROSSPROFITDEADNET'] = df['STRATA_SUM_GROSSPROFITDEADNET'].astype(float)
df['INDEX_SUMPROD'] = df['INDEX_SUMPROD'].astype(float)

In [18]:
df.to_csv('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\product_to_zip_mapping.csv', index=False)
df.to_pickle('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\product_to_zip_mapping.pkl')

In [19]:
print(df.shape)
print(df.columns.values)
df.head()

(1425637, 30)
['CS_UPC' 'EA_UPC' 'EAN_UPC' 'SUPER_CHANNEL_DEFINITION' 'Zip' 'SUM_VOLUME'
 'SUM_DEADNET' 'SUM_GROSSPROFITDEADNET' 'SUM_NSI' 'MATERIAL_DESCRIPTION'
 'BEV_PRODUCT_DESC' 'BRAND_DESC' 'SWIRE_PACKAGE_CATEGORY_DESCRIPTION'
 'PACK_SIZE_SALES_UNIT_DESCRIPTION' 'PACK_TYPE_DESC' 'CALORIE_CAT_DESC'
 'IC_FC_ATTRIBUTE' 'FLAVOUR_DESC' 'AU_SUM_VOLUME' 'AU_ROW_COUNTS'
 'STRATA_SUM_VOLUME' 'STRATA_SUM_GROSSPROFITDEADNET' 'STRATA_ROW_COUNTS'
 'ZIP_SC_GROUP' 'ZIP_SC_PERCENTAGE' 'FOOTPRINT_SC_PROD'
 'FOOTPRINT_SC_GROUP' 'FOOTPRINT_SC_PERCENTAGE' 'PRODUCT_INDEX'
 'INDEX_SUMPROD']


,CS_UPC,EA_UPC,EAN_UPC,SUPER_CHANNEL_DEFINITION,Zip,SUM_VOLUME,SUM_DEADNET,SUM_GROSSPROFITDEADNET,SUM_NSI,MATERIAL_DESCRIPTION,...,STRATA_SUM_VOLUME,STRATA_SUM_GROSSPROFITDEADNET,STRATA_ROW_COUNTS,ZIP_SC_GROUP,ZIP_SC_PERCENTAGE,FOOTPRINT_SC_PROD,FOOTPRINT_SC_GROUP,FOOTPRINT_SC_PERCENTAGE,PRODUCT_INDEX,INDEX_SUMPROD
0,25000058011,25000058011,25000058011,CONVENIENCE RETAIL,57551,2.0000000000000000,48.1600000000000000,23.9900000000000000,48.1600000000000000,20Z PT 24LS_MM LMND,...,2.0,23.99,1,107.0000000000000000,0.01869158878504672897196261682,65563.7080000000000000,30873307.3090000000000000,0.002123637333175745120167876991,101.656795,203.313590
1,25000061523,25000061516,25000061523,CONVENIENCE RETAIL,57551,2.0000000000000000,62.3800000000000000,23.4000000000000000,62.3800000000000000,12Z PT 24LS_MM JTG APL J100,...,2.0,23.40,1,107.0000000000000000,0.01869158878504672897196261682,98284.1680000000000000,30873307.3090000000000000,0.003183467421106153833089486189,101.550812,203.101624
2,25000062193,25000061530,25000062193,CONVENIENCE RETAIL,57551,3.0000000000000000,86.1300000000000000,26.2400000000000000,86.1300000000000000,12Z PT 24LS_MM JTG OJ100,...,3.0,26.24,1,107.0000000000000000,0.02803738317757009345794392523,98724.0820000000000000,30873307.3090000000000000,0.003197716429014411168669004227,102.483967,307.451900
3,49000023190,49000023190,49000023190,CONVENIENCE RETAIL,57551,5.0000000000000000,132.0000000000000000,85.8000000000000000,132.0000000000000000,1L PT 12LS SH_SPRITE,...,5.0,85.80,1,107.0000000000000000,0.04672897196261682242990654206,186378.1680000000000000,30873307.3090000000000000,0.006036870819656829011742727638,104.069210,520.346051
4,49000028904,49000006346,49000028904,CONVENIENCE RETAIL,57551,10.0000000000000000,137.0800000000000000,53.5100000000000000,137.0800000000000000,12Z CN 12FP_COKE,...,10.0,53.51,1,107.0000000000000000,0.09345794392523364485981308411,1576197.7880000000000000,30873307.3090000000000000,0.05105373947223711742570145516,104.240420,1042.404204


In [20]:
# Load demographics data

In [21]:
# Helper function to read in the demographic spreadsheet for a state
def read_and_clean_single_file(filename):
    df = pd.read_excel(filename, sheet_name='Table', skiprows=12, header=[0,1], index_col=0)
    df = df.iloc[:-2,:].drop(df.columns[0:2], axis=1) # Discard first two columns and last two rows
    return df

In [22]:
# use glob to get all the xls files  in the folder
path = "C:\\DemographicsData"
spectra_excel_files = glob.glob(os.path.join(path, "*.xls"))
df_spectra = pd.DataFrame()

# loop over the list of xlsx files
for f in spectra_excel_files:
    df_spectra_this_iteration = read_and_clean_single_file(f)    
    df_spectra=df_spectra.append(df_spectra_this_iteration,ignore_index=False)
    
print(df_spectra.shape)

(6552, 130)


In [23]:
# Split out age and gender
df_gender1 = df_spectra.filter(regex = 'Males', axis=1)
df_gender1_counts = df_gender1.loc[:, (slice(None), "Adults")].copy()
df_gender1_counts.columns = df_gender1_counts.columns.droplevel(1)
df_gender1_counts['Males'] = df_gender1_counts.sum(axis=1)
df_gender1_counts['Females'] = 0
df_gender2 = df_spectra.filter(regex = 'Females', axis=1)
df_gender2_counts = df_gender2.loc[:, (slice(None), "Adults")].copy()
df_gender2_counts.columns = df_gender2_counts.columns.droplevel(1)
df_gender2_counts['Females'] = df_gender2_counts.sum(axis=1)
df_gender2_counts['Males'] = 0

mdict = {
    'Males 18 - 24':'Age 18 - 24',
    'Males 25 - 34':'Age 25 - 34',
    'Males 35 - 44':'Age 35 - 44',
    'Males 45 - 54':'Age 45 - 54',
    'Males 55 - 64':'Age 55 - 64',
    'Males 65 - 74':'Age 65 - 74',
    'Males 75 or More':'Age 75 or More'
}

fdict = {
    'Females 18 - 24':'Age 18 - 24',
    'Females 25 - 34':'Age 25 - 34',
    'Females 35 - 44':'Age 35 - 44',
    'Females 45 - 54':'Age 45 - 54',
    'Females 55 - 64':'Age 55 - 64',
    'Females 65 - 74':'Age 65 - 74',
    'Females 75 or More':'Age 75 or More'
}

df_gender1_counts = df_gender1_counts.rename(columns=mdict)
df_gender2_counts = df_gender2_counts.rename(columns=fdict)
df_age_counts = df_gender1_counts + df_gender2_counts
df_age_counts.columns = pd.MultiIndex.from_product([df_age_counts.columns, ['Adults']])
print(df_age_counts.shape)
df_age_counts.head()

(6552, 9)


Geography Name,Age 18 - 24,Age 25 - 34,Age 35 - 44,Age 45 - 54,Age 55 - 64,Age 65 - 74,Age 75 or More,Females,Males
,Adults,Adults,Adults,Adults,Adults,Adults,Adults,Adults,Adults
"85308 Glendale, AZ",5598,10432,8815,8684,10140,7289,3735,28182,26511
"85142 Queen Creek, AZ",7430,10171,10052,10331,7269,4867,2664,26999,25785
"85383 Peoria, AZ",5911,7959,7230,8723,8668,5326,2203,23464,22556
"85255 Scottsdale, AZ",4102,4869,4267,6611,8393,6821,4414,20597,18880
"85032 Phoenix, AZ",5449,10244,10329,9228,8656,6130,3948,27366,26618


In [24]:
dfn = df_spectra.filter(regex = 'Males|Females', axis=1)
dfu = df_spectra[list(set(df_spectra.columns.values) - set(dfn.columns.values))]
dfa = pd.concat([dfu,df_age_counts], axis=1)

Criteria = dfa.columns.get_level_values(0)
Units = dfa.columns.get_level_values(1)
Criteria_Groups = np.empty(len(Criteria), dtype=object)

In [25]:
# Define criteria groups
l = ['Age']
Criteria_Groups[Criteria.str.contains(pat='|'.join(l))] = 'Age'
l = ['Race']
Criteria_Groups[Criteria.str.contains(pat='|'.join(l))] = 'Ethnicity'
l = ['Males','Females']
Criteria_Groups[Criteria.str.contains(pat='|'.join(l))] = 'Gender'
l = ['Single','Married','Divorced']
Criteria_Groups[Criteria.str.contains(pat='|'.join(l))] = 'Marital Status'
l = ['School','College']
Criteria_Groups[Criteria.str.contains(pat='|'.join(l))] = 'Education'
l = ['Employment','Military','Unemployed','Retired','Labor']
Criteria_Groups[Criteria.str.contains(pat='|'.join(l))] = 'Employment'
l = ['Urban','Countrysides','Cities']
Criteria_Groups[Criteria.str.contains(pat='|'.join(l))] = 'Lifestyle'
l = ['Person']
Criteria_Groups[Criteria.str.contains(pat='|'.join(l))] = 'Size of HH'
l = ['Income']
Criteria_Groups[Criteria.str.contains(pat='|'.join(l))] = 'Income'

dfa.columns = pd.MultiIndex.from_arrays([Criteria,Units,Criteria_Groups])

In [26]:
# Reshape data so each row is a zip and criteria 
col_names_list=dfa.columns.to_list()
df_spectra2=pd.melt(dfa, value_vars=col_names_list, ignore_index=False).reset_index()
df_spectra2.columns=['Geography Name','Consumer_Criteria','Criteria_Units','Consumer_Criteria_Group','ZIP_CRITERIA_COUNT']
df_spectra2.loc[df_spectra2['Criteria_Units'].str.contains('Index'),'Criteria_Units'] = 'Index'
df_spectra2['Zip'] = df_spectra2['Geography Name'].str.slice(0,5)
print(len(df_spectra2['Zip'].unique()))

6552


In [27]:
# Restrict to zips within Swire footprint
df_swire_sales = pd.read_pickle('C:\\CodeShare\\DataPull2022\\df_swire_sales_2022.pkl') # can omit if already defined above
zips = df_swire_sales['ADDRESS_ZIP_CODE'].str.slice(0,5).unique()
df_spectra2 = df_spectra2[df_spectra2['Zip'].isin(zips)]
print(len(df_spectra2['Zip'].unique()))
df_spectra2_indices = df_spectra2[df_spectra2['Criteria_Units'] == 'Index'].reset_index(drop=True)
df_spectra2 = df_spectra2[df_spectra2['Criteria_Units'] != 'Index'].reset_index(drop=True)

2129


In [28]:
# Calculate percentages within each zip and criteria
dfg = df_spectra2.groupby(['Zip','Consumer_Criteria_Group'])['ZIP_CRITERIA_COUNT'].sum().reset_index(name='ZIP_CRITERIA_GROUP_COUNT')
df_spectra2 = df_spectra2.merge(dfg, on=['Zip','Consumer_Criteria_Group'], how='inner')
df_spectra2['ZIP_CRITERIA_PERCENTAGE'] = df_spectra2['ZIP_CRITERIA_COUNT']/df_spectra2['ZIP_CRITERIA_GROUP_COUNT']

In [29]:
# Calculate percentages within each criteria, across Swire footprint zips
dfga1 = df_spectra2.groupby(['Consumer_Criteria_Group'])['ZIP_CRITERIA_COUNT'].sum().reset_index(name='FOOTPRINT_CRITERIA_GROUP_COUNT')
dfga2 = df_spectra2.groupby(['Consumer_Criteria','Consumer_Criteria_Group'])['ZIP_CRITERIA_COUNT'].sum().reset_index(name='FOOTPRINT_CRITERIA_COUNT')
dfga3 = dfga2.merge(dfga1, on='Consumer_Criteria_Group', how='inner')
dfga3['FOOTPRINT_CRITERIA_PERCENTAGE'] = dfga3['FOOTPRINT_CRITERIA_COUNT']/dfga3['FOOTPRINT_CRITERIA_GROUP_COUNT']

In [30]:
# Calculate index as percentage within this zip/criteria minus percentage within criteria across Swire footprint + 100
df_spectra2 = df_spectra2.merge(dfga3, on=['Consumer_Criteria','Consumer_Criteria_Group'], how='inner').reset_index(drop=True)
df_spectra2 = df_spectra2.sort_values(by=['Zip','Consumer_Criteria_Group']).reset_index(drop=True)
df_spectra2['CONSUMER_ATTRIBUTE_INDEX'] =100*(df_spectra2['ZIP_CRITERIA_PERCENTAGE']-df_spectra2['FOOTPRINT_CRITERIA_PERCENTAGE'])+100
df_spectra2['CONSUMER_ATTRIBUTE_INDEX_INTO_COUNT'] = df_spectra2['CONSUMER_ATTRIBUTE_INDEX']*df_spectra2['ZIP_CRITERIA_COUNT']

In [31]:
# Recast national indices to be similarly percentage difference + 100 rather than ratios
df_spectra2_indices = df_spectra2_indices.rename(columns={'ZIP_CRITERIA_COUNT':'ZIP_INDEX'})
df_spectra2_indices = df_spectra2_indices[['Consumer_Criteria','Consumer_Criteria_Group','ZIP_INDEX','Zip']]
df_spectra2_indices = df_spectra2_indices.merge(df_spectra2, on=['Zip','Consumer_Criteria','Consumer_Criteria_Group'], how='inner')
df_spectra2_indices['ZIP_INDEX_DIFF'] = 100*(df_spectra2_indices['ZIP_CRITERIA_PERCENTAGE'] - df_spectra2_indices['ZIP_CRITERIA_PERCENTAGE']/(df_spectra2_indices['ZIP_INDEX']/100)) + 100
df_spectra2_indices = df_spectra2_indices[['Zip','Consumer_Criteria','Consumer_Criteria_Group','ZIP_CRITERIA_COUNT',
                             'CONSUMER_ATTRIBUTE_INDEX','CONSUMER_ATTRIBUTE_INDEX_INTO_COUNT', 'ZIP_INDEX_DIFF']]

In [32]:
# Load age and gender percentages from US census data

dfu = pd.read_excel('C:\\CodeShare\\DataPull2022\\agesex2021_table1.xlsx', skiprows=4)
dfu = dfu.iloc[1:]
dfu.columns = ['Age','Both_count','Both_perc','Male_count','Male_perc','Female_count','Female_perc']
dfu=dfu[~dfu['Both_count'].isna()]
dfu['Ages'] = ['','','','','','Age 18 - 24','Age 25 - 34','Age 25 - 34',
              'Age 35 - 44','Age 35 - 44','Age 45 - 54','Age 45 - 54',
              'Age 55 - 64','Age 55 - 64','Age 65 - 74','Age 65 - 74',
              'Age 75 or More','Age 75 or More','Age 75 or More',
               '','','','Age 18 - 24','','','','']
# scale 18-20 by 2/3 so it is 18-19
dfu.loc[dfu['Age'].str.strip() == '.18 to 20 years','Both_count'] = 0.67*dfu.loc[dfu['Age'].str.strip() == '.18 to 20 years','Both_count']
dfu.loc[dfu['Age'].str.strip() == '.18 to 20 years','Male_count'] = 0.67*dfu.loc[dfu['Age'].str.strip() == '.18 to 20 years','Male_count']
dfu.loc[dfu['Age'].str.strip() == '.18 to 20 years','Female_count'] = 0.67*dfu.loc[dfu['Age'].str.strip() == '.18 to 20 years','Female_count']

dfuu = dfu[['Ages','Both_count','Male_count','Female_count']]
dfuu = dfuu[dfuu['Ages'] != '']
dfgu = dfuu.groupby('Ages')[['Both_count','Male_count','Female_count']].sum().reset_index()
dfgu['Both_perc'] = dfgu['Both_count']/np.sum(dfgu['Both_count'])
dfgu['Male_perc'] = dfgu['Male_count']/np.sum(dfgu['Male_count'])
dfgu['Female_perc'] = dfgu['Female_count']/np.sum(dfgu['Female_count'])

dfgus = dfgu[['Ages','Both_perc']].rename(columns={'Ages':'Consumer_Criteria','Both_perc':'USA_perc'})
dfgus['Consumer_Criteria_Group'] = 'Age'
tot = dfgu['Male_count'].sum() + dfgu['Female_count'].sum()
dfgen = pd.DataFrame(data=[['Males','Gender',dfgu['Male_count'].sum()/tot],
                           ['Females','Gender',dfgu['Female_count'].sum()/tot]],
             columns=['Consumer_Criteria','Consumer_Criteria_Group','USA_perc'])
dfagegen = pd.concat([dfgus,dfgen])

In [33]:
# Add age and gender national indices; note these may not be as accurate or fully consistent 
dfagegen_zip = df_spectra2.merge(dfagegen, on=['Consumer_Criteria','Consumer_Criteria_Group'], how='inner')
dfagegen_zip['ZIP_INDEX_DIFF'] = 100*(dfagegen_zip['ZIP_CRITERIA_PERCENTAGE'] - dfagegen_zip['USA_perc']) + 100
dfagegen_zip = dfagegen_zip[['Zip','Consumer_Criteria','Consumer_Criteria_Group','ZIP_CRITERIA_COUNT',
                             'CONSUMER_ATTRIBUTE_INDEX','CONSUMER_ATTRIBUTE_INDEX_INTO_COUNT', 'ZIP_INDEX_DIFF']]
df_spectra2_indices = pd.concat([dfagegen_zip,df_spectra2_indices])

In [34]:
income_dict = {'Income Under $10,000':'01-Income Under $10,000',
               'Income $10,000 - $19,999':'02-Income $10,000 - $19,999',
               'Income $20,000 - $29,999':'03-Income $20,000 - $29,999',
               'Income $30,000 - $39,999':'04-Income $30,000 - $39,999',
               'Income $40,000 - $49,999':'05-Income $40,000 - $49,999',
               'Income $50,000 - $74,999':'06-Income $50,000 - $74,999',
               'Income $75,000 - $99,999':'07-Income $75,000 - $99,999',
               'Income $100,000 - $149,999':'08-Income $100,000 - $149,999',
               'Income $150,000 or More':'09-Income $150,000 or More'}

df_spectra2_indices['Consumer_Criteria'] = df_spectra2_indices['Consumer_Criteria'].map(income_dict).fillna(df_spectra2_indices['Consumer_Criteria'])

df_spectra2_indices['Consumer_Criteria'].unique()

array(['Age 18 - 24', 'Age 25 - 34', 'Age 35 - 44', 'Age 45 - 54',
       'Age 55 - 64', 'Age 65 - 74', 'Age 75 or More', 'Females', 'Males',
       'Striving Countrysides', '4 Person Households',
       '08-Income $100,000 - $149,999', 'Race: White (Non-Hispanic)',
       'Full Time Civilian Employment', 'College Graduate',
       'Single (Never Married)', 'Downscale Urban',
       '09-Income $150,000 or More', '01-Income Under $10,000',
       'Wealthy Urban', 'Grade School Education',
       'Downscale Sprawling Cities/Suburbs',
       'Affluent Sprawling Cities/Suburbs', 'Lower Middle Countrysides',
       'Wealthy Countrysides', '03-Income $20,000 - $29,999',
       'Striving Urban', 'Retired', 'Race: Other (Non-Hispanic)',
       '2 Person Households', 'Affluent Urban',
       'Wealthy Sprawling Cities/Suburbs', '04-Income $30,000 - $39,999',
       'Race: Asian (Non-Hispanic)', 'Divorced, Separated & Widowed',
       'Race: Hispanic', 'Striving Sprawling Cities/Suburbs',
       

In [35]:
df_spectra2_indices.to_csv('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\df_spectra2_indices.csv', index=False) 
df_spectra2_indices.to_pickle('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\df_spectra2_indices.pkl')

In [36]:
print(df_spectra2_indices.shape)
print(df_spectra2_indices.columns.values)
df_spectra2_indices.head()

(127740, 7)
['Zip' 'Consumer_Criteria' 'Consumer_Criteria_Group' 'ZIP_CRITERIA_COUNT'
 'CONSUMER_ATTRIBUTE_INDEX' 'CONSUMER_ATTRIBUTE_INDEX_INTO_COUNT'
 'ZIP_INDEX_DIFF']


,Zip,Consumer_Criteria,Consumer_Criteria_Group,ZIP_CRITERIA_COUNT,CONSUMER_ATTRIBUTE_INDEX,CONSUMER_ATTRIBUTE_INDEX_INTO_COUNT,ZIP_INDEX_DIFF
0,57551,Age 18 - 24,Age,245,103.855246,25444.535178,103.667576
1,57702,Age 18 - 24,Age,2739,97.929111,268227.834957,97.741441
2,57714,Age 18 - 24,Age,66,104.655124,6907.238171,104.467454
3,57716,Age 18 - 24,Age,57,103.792460,5916.170214,103.604790
4,57752,Age 18 - 24,Age,216,106.146509,22927.645915,105.958839


In [37]:
# store information with volume quartiles

In [4]:
scol = ['CUSTOMER_NUMBER', 'CUSTOMER_NAME', 'KEY_ACCOUNT_PLAN_DESCRIPTION_CP3', 'DIVISION', 
        'ADDRESS_CITY', 'ADDRESS_ZIP_CODE', 'ADDRESS_STATE', 'SUPER_CHANNEL_DEFINITION']
dfs = df_swire_sales[scol + ['TOT_VOLUME']]
dfs = dfs.groupby(scol)['TOT_VOLUME'].sum().reset_index(name='SUM_TOT_VOLUME')
dfs['Zip'] = dfs['ADDRESS_ZIP_CODE'].str.slice(0,5)
dfs = dfs.drop(columns='ADDRESS_ZIP_CODE')
dfs['SUM_TOT_VOLUME'] = dfs['SUM_TOT_VOLUME'].astype(float)
print(dfs.shape)
print(dfs.columns.values)
dfs.head()

(66708, 9)
['CUSTOMER_NUMBER' 'CUSTOMER_NAME' 'KEY_ACCOUNT_PLAN_DESCRIPTION_CP3'
 'DIVISION' 'ADDRESS_CITY' 'ADDRESS_STATE' 'SUPER_CHANNEL_DEFINITION'
 'SUM_TOT_VOLUME' 'Zip']


,CUSTOMER_NUMBER,CUSTOMER_NAME,KEY_ACCOUNT_PLAN_DESCRIPTION_CP3,DIVISION,ADDRESS_CITY,ADDRESS_STATE,SUPER_CHANNEL_DEFINITION,SUM_TOT_VOLUME,Zip
0,0500245678,FOOD N STUFF,ALL OTHER PLAN KA,UT,KAMAS,UT,ON PREMISE,86.0,84036
1,0500245685,SLAB PIZZA,ALL OTHER PLAN KA,UT,LEHI,UT,ON PREMISE,24.0,84043
2,0500245688,TARAHUMARA OF PARK CITY,ALL OTHER PLAN KA,UT,PARK CITY,UT,ON PREMISE,170.0,84060
3,0500245689,THE WESTERNER,ALL OTHER PLAN KA,UT,WEST VALLEY CITY,UT,ON PREMISE,125.0,84119
4,0500245690,TERIYAKI GRILL-SPANISH FORK,ALL OTHER PLAN KA,UT,SPANISH FORK,UT,ON PREMISE,91.0,84660


In [5]:
dfsx = dfs.sort_values(by='SUM_TOT_VOLUME')
dfsx['RUNNING_SUM_VOL'] = dfsx['SUM_TOT_VOLUME'].cumsum()
quartiles = np.max(dfsx['RUNNING_SUM_VOL'])/4
dfsx['VOLUME_QUARTILE'] = np.where(dfsx['RUNNING_SUM_VOL'] <= quartiles*1, 4, 
                                    np.where(dfsx['RUNNING_SUM_VOL'] <= quartiles*2, 3,
                                            np.where(dfsx['RUNNING_SUM_VOL'] <= quartiles*3, 2,
                                                    np.where(dfsx['RUNNING_SUM_VOL'] <= quartiles*4, 1, 0))))

In [6]:
dfsx.to_csv('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\stores_with_volumes.csv', index=False)
dfsx.to_pickle('C:\Users\ddeming\OneDrive - Swire Coca-Cola, USA\Documents\Projects\streamlit\NLI_Streamlit\Customer_Cube_App\accessories\stores_with_volumes.pkl')